# Setup

## Imports

In [1]:
import torch

TORCH_VERSION = torch.__version__[:5]
CUDA_VERSION = torch.version.cuda.replace('.','')
print(f'Running torch version {TORCH_VERSION}, with CUDA version {CUDA_VERSION}')

Running torch version 1.8.0, with CUDA version 101


In [7]:
# deep learning
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim
import numpy as np
torch.set_printoptions(threshold=10000)

# meshes
import trimesh

# pytorch geometric
import torch_geometric
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data, Batch
from torch_geometric.data import DataLoader
from torch_geometric.utils import dropout_adj
from torch_geometric.utils.convert import from_networkx
import torch_geometric.transforms as T 
from torch_sparse import SparseTensor, set_diag

import torch
import torch.nn as nn
import numpy as np
import scipy
from scipy import sparse
import torch.utils.data as data
import scipy.sparse.linalg
from scipy.sparse import coo_matrix
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import pandas as pd

# graphs
import networkx as nx

# misc
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import *
from typing import *
import json
import os
import math
import random
import dill

In [8]:
import torch_geometric

## Configuration

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
PROJECT_HOME = ".."
data_folder = os.path.join(PROJECT_HOME, f'data')

## Run settings

In [11]:
use_dummy_data = False

# Functions

## Normal of a triangle

\begin{align}
Nx &= Ay * Bz - Az * By \\
Ny &= Az * Bx - Ax * Bz \\
Nz &= Ax * By - Ay * Bx \\
\end{align}

In [12]:
def compute_normal(triangle):
    assert triangle.shape[0] == 3
    v1, v2, v3 = triangle
    
    A = v2 - v1
    B = v3 - v1

    N = np.zeros(3, dtype=np.float64)
    
    N[0] = A[1] * B[2] - A[2] * B[1]
    N[1] = A[2] * B[0] - A[0] * B[2]
    N[2] = A[0] * B[1] - A[1] * B[0]  

    norm = np.sqrt(N[0]**2 + N[1]**2 + N[2]**2)
    
    return N/norm

# Data

## Dummy mesh

In [13]:
triangles = torch.tensor([ 
                          [0, 1, 2],
                          [1, 2, 3],
                          [2, 4, 5]
                          ])

positions = torch.tensor( [ 
                           [3, 2, 0],
                           [5, 2, 0],
                           [4, 4, 0],
                           [6, 4, 0],
                           [3, 6, 0],
                           [2, 4, 0]
                           ])

In [14]:
num_nodes = positions.shape[0]
num_triangles = triangles.shape[0]
num_edges = 8
print(f'There are {num_nodes} nodes and {num_triangles} triangles')

There are 6 nodes and 3 triangles


## Data loading from file

In [14]:
mesh_name = 'bob'

### Mesh loading

In [15]:
mesh = trimesh.load(os.path.join(data_folder, f'{mesh_name}_tri.obj'))

ValueError: ignored

In [ ]:
print(mesh)
triangles = mesh.faces
positions = mesh.vertices
num_triangles = triangles.shape[0]
num_vertices = positions.shape[0]

### Plot

In [ ]:
import plotly.figure_factory as ff
import numpy as np
from scipy.spatial import Delaunay


x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]

fig = ff.create_trisurf(x=x, y=y, z=z,
                         simplices=triangles,
                         title=mesh_name, aspectratio=dict(x=1, y=1, z=0.3))
fig.show()

## Data loading from PyG

### Mesh loading

In [ ]:
!pip install openmesh

     |████████████████████████████████| 9.1 MB 3.6 MB/s eta 0:00:01

In [16]:
mesh = torch_geometric.datasets.FAUST(data_folder)

Processing...


ImportError: `read_ply` requires the `openmesh` package.

In [ ]:
print(mesh)
triangles = mesh.faces
positions = mesh.vertices
num_triangles = triangles.shape[0]
num_vertices = positions.shape[0]

### Plot

In [ ]:
import plotly.figure_factory as ff
import numpy as np
from scipy.spatial import Delaunay


x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]

fig = ff.create_trisurf(x=x, y=y, z=z,
                         simplices=triangles,
                         title=mesh_name, aspectratio=dict(x=1, y=1, z=0.3))
fig.show()

## Data sorting

Define a canonical ordering

In [ ]:
triangles = sorted(triangles, key=lambda tr: (tr[0], tr[1], tr[2]))
print(triangles)

# Adding noise

In [ ]:
print(positions)
sigma = 10
noisy_positions = np.random.normal(positions, sigma)
print(noisy_positions)

[[ 0.687488  -0.0813099 -0.389237 ]
 [ 0.687488  -0.0813099  0.389237 ]
 [-0.279168  -0.509094   0.0604992]
 ...
 [-0.0214861 -0.492915  -0.349771 ]
 [-0.0242509 -0.271801  -0.156825 ]
 [-0.0242509 -0.271801   0.156825 ]]
[[ -3.55936734  -5.84102056 -19.44114272]
 [-18.76973007  -4.81566988  -3.93059377]
 [ -5.94144547 -12.58811178  -3.57023499]
 ...
 [ -9.88617752   5.99593781  -3.60583637]
 [  9.09820865  -0.57524936  -3.04371078]
 [  0.67520276   2.04144994  14.29910746]]


# Simplices creation

## Create simplices from mesh

In [ ]:
def create_simplices(triangles):
    """
    input:
        triangles: list of triangles T_1, ..., T_n where each T_i is a tensor (3, )
                   containing the indices of the nodes that compose the triangle
    return:
        simplices: dict of dicts, simplices[d] is a dict with the d-dimensional simplices as keys and their corresponding ids as value
                   e.g. dict[1] has edges {n_i, n_j} as keys and their corresponding id in the laplacian and boundary matrices as values  
    """
    simplices = {0: {}, 1: {}, 2: {}}

    for triangle in triangles:
        
        nodes = [ node.item() for node in triangle]
        node_0, node_1, node_2 = nodes

        # need to wrap the simplex in a frozenset to use it as a dict key
        triangle_simplex = frozenset([node_0, node_1, node_2])
        
        # assign a progressive id to the triangles
        simplices[2][triangle_simplex] = len(simplices[2])

        # all the edges composing the triangle
        edges = [ (node_0, node_1), (node_0, node_2), (node_1, node_2) ]
        
        # give a progressive id to unseen edges
        for edge in edges:
            edge_simplex = frozenset(edge)
            if edge_simplex not in simplices[1]:
                simplices[1][edge_simplex] = len(simplices[1])

        # give a progressive id to unseen nodes
        for node in nodes:
            node_simplex = frozenset({node})
            if node_simplex not in simplices[0]:
                simplices[0][node_simplex] = len(simplices[0])
    
    return simplices

In [ ]:
simplices = create_simplices(triangles)

Print some simplices

In [ ]:
for d in range(3):
    print(list(simplices[d].items())[0:10])

[(frozenset({0}), 0), (frozenset({3380}), 1), (frozenset({3616}), 2), (frozenset({4377}), 3), (frozenset({421}), 4), (frozenset({1}), 5), (frozenset({422}), 6), (frozenset({4378}), 7), (frozenset({3615}), 8), (frozenset({3379}), 9)]
[(frozenset({0, 3380}), 0), (frozenset({0, 3616}), 1), (frozenset({3616, 3380}), 2), (frozenset({0, 4377}), 3), (frozenset({0, 421}), 4), (frozenset({4377, 421}), 5), (frozenset({1, 422}), 6), (frozenset({1, 4378}), 7), (frozenset({4378, 422}), 8), (frozenset({1, 3615}), 9)]
[(frozenset({0, 3616, 3380}), 0), (frozenset({0, 4377, 421}), 1), (frozenset({1, 4378, 422}), 2), (frozenset({1, 3379, 3615}), 3), (frozenset({2, 1618, 2718}), 4), (frozenset({2, 2858, 1972}), 5), (frozenset({1617, 3, 2719}), 6), (frozenset({2857, 3, 1971}), 7), (frozenset({4455, 4, 2839}), 8), (frozenset({392, 3283, 4}), 9)]


# Signal creation

## Create signals from mesh

In [ ]:
def create_signals(triangles, positions):
    """
    input:
        triangles: list of triangles T_1, ..., T_n where each T_i is a tensor (3, )
                   containing the indices of the nodes that compose the triangle
    return:
        signals: dict of dicts, signals[0] contains the node positions, signal[2] contains the triangle normals 
    """

    node_signals, triangle_signals = {}, {}

    for triangle in triangles:
        
        nodes = [ node.item() for node in triangle]

        # the signal for each node is its position (x, y, z)
        for node in nodes:
            node_signals[frozenset({node})] = positions[node]

        # the signal for each triangle is its normal (n_x, n_y, n_z)
        triangle_normal = compute_normal(positions[nodes])
        
        triangle_signals[frozenset(nodes)] = triangle_normal
    
    return node_signals, triangle_signals

In [ ]:
node_signals, triangle_signals = create_signals(triangles, positions)
noisy_node_signals, _ = create_signals(triangles, noisy_positions)

In [ ]:
assert len(triangle_signals) == num_triangles
print(list(node_signals.items())[0:10])
print(list(triangle_signals.items())[0:10])

[(frozenset({0}), TrackedArray([ 0.687488 , -0.0813099, -0.389237 ])), (frozenset({3380}), TrackedArray([ 0.712342 , -0.0805019, -0.356168 ])), (frozenset({3616}), TrackedArray([ 0.706065, -0.102015, -0.403346])), (frozenset({4377}), TrackedArray([ 0.660802 , -0.0815793, -0.421255 ])), (frozenset({421}), TrackedArray([ 0.66698  , -0.0638557, -0.373593 ])), (frozenset({1}), TrackedArray([ 0.687488 , -0.0813099,  0.389237 ])), (frozenset({422}), TrackedArray([ 0.66698  , -0.0638557,  0.373593 ])), (frozenset({4378}), TrackedArray([ 0.660802 , -0.0815793,  0.421255 ])), (frozenset({3615}), TrackedArray([ 0.706065, -0.102015,  0.403346])), (frozenset({3379}), TrackedArray([ 0.712342 , -0.0805019,  0.356168 ]))]
[(frozenset({0, 3616, 3380}), array([ 0.52179101,  0.74784546, -0.41044038])), (frozenset({0, 4377, 421}), array([ 0.42747172,  0.82783909, -0.36324974])), (frozenset({1, 4378, 422}), array([0.42747172, 0.82783909, 0.36324974])), (frozenset({1, 3379, 3615}), array([0.52179101, 0.747

# Create graph from mesh

In [ ]:
g = nx.Graph()

In [ ]:
for triangle in triangles:
    
    nodes = [ node.item() for node in triangle]
    node_0, node_1, node_2 = nodes
    edges = [ (node_0, node_1), (node_1, node_2), (node_2, node_0)]
    
    for edge in edges:
        g.add_edge(edge[0], edge[1])

In [ ]:
node_positions_mapping = { node: {'x': pos[0].item(), 'y': pos[1].item(), 'z': pos[2].item()} for node, pos in enumerate(positions) }

In [ ]:
nx.set_node_attributes(g, node_positions_mapping)

In [ ]:
# nx.draw(g)

# Create incidence matrices

In [ ]:
def build_boundaries(simplices):
    """
    Build the boundary operators from a list of simplices.

    Parameters
    ----------
    simplices:  
                List of dictionaries, one per dimension d. 
                The size of the dictionary is the number of d-simplices.
                The dictionary's keys are sets (of size d+1) of the vertices that constitute the d-simplices.
                The dictionary's values are the indexes of the simplices in the boundary and Laplacian matrices.
    Returns
    -------
    boundaries: 
                List of boundary operators, one per dimension: i-th boundary is in (i-1)-th position
    """
    boundaries = list()

    for dim in range(1, len(simplices)):
        idx_simplices, idx_faces, values = [], [], []

        # simplex is a frozenset of vertices, idx_simplex is the integer progressive id of the simplex
        for simplex, idx_simplex in simplices[dim].items():
            simplices_list_sorted = np.sort(list(simplex))
            
            for i, left_out in enumerate(simplices_list_sorted):
                # linear combination of the face of the simplex obtained by removing
                # the i-th vertex
                idx_simplices.append(idx_simplex)
                values.append((-1)**i)
                face = simplex.difference({left_out})
                idx_faces.append(simplices[dim-1][face])
                
        assert len(values) == (dim+1) * len(simplices[dim])
        boundary = coo_matrix((values, (idx_faces, idx_simplices)),
                                     dtype=np.float32,
                                     shape=(len(simplices[dim-1]), len(simplices[dim])))
        boundaries.append(boundary)
    return boundaries

In [ ]:
boundaries = build_boundaries(simplices)
print(boundaries[0].todense())

[[-1. -1.  0. ...  0.  0.  0.]
 [ 1.  0. -1. ...  0.  0.  0.]
 [ 0.  1.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


# Create Laplacians

In [ ]:
def build_laplacians(boundaries):
    """Build the Laplacian operators from the boundary operators.

    Parameters
    ----------
    boundaries: list of sparse matrices
       List of boundary operators, one per dimension.

    Returns
    -------
    laplacians: list of sparse matrices
       List of Laplacian operators, one per dimension: laplacian of degree i is in the i-th position
    """
    laplacians = list()
    # graph Laplacian L0
    upper = coo_matrix(boundaries[0] @ boundaries[0].T)
    laplacians.append(upper)

    for dim in range(len(boundaries)-1):
        # lower Laplacian B_{k}^T B_k 
        lower = boundaries[dim].T @ boundaries[dim]
        # upper Laplacian B_{k+1} B_{k}^T 
        upper = boundaries[dim+1] @ boundaries[dim+1].T
        # L_k = L_k_lower + L_k_upper
        laplacians.append(coo_matrix(lower + upper))

    # last Laplacian L_K
    lower = boundaries[-1].T @ boundaries[-1]
    laplacians.append(coo_matrix(lower))
    return laplacians

In [ ]:
laplacians = build_laplacians(boundaries)

# Save

In [ ]:
def save_dict(dictionary, path):
    keys = list(dictionary.keys())
    
    values = [tens for tens in list(dictionary.values())]
    values = np.array(values)
    keys_path = path + '_keys'
    values_path = path + '_values'

    with open(keys_path, 'wb+') as f:
        dill.dump(keys, f)
    np.save(values_path, values)

In [ ]:
prefix = 'dummy_' if use_dummy_data else ''
laplacian_path = f'{data_folder}/{prefix}laplacians.npy'
boundaries_path = f'{data_folder}/{prefix}boundaries.npy'
positions_path = f'{data_folder}/{prefix}positions'
noisy_positions_path = f'{data_folder}/{prefix}noisy_positions'
original_positions_path = f'{data_folder}/{prefix}original_positions'
normals_path = f'{data_folder}/{prefix}normals'
triangles_path = f'{data_folder}/{prefix}triangles.npy'

In [ ]:
np.save(laplacian_path, laplacians)
np.save(boundaries_path, boundaries)    

In [ ]:
with open(triangles_path, 'wb+') as f:
    dill.dump(triangles, f)

with open(original_positions_path, 'wb+') as f:
    dill.dump(positions, f)

In [ ]:
save_dict(node_signals, positions_path)
save_dict(triangle_signals, normals_path)
save_dict(noisy_node_signals, noisy_positions_path)